In [14]:
using Pkg
Pkg.activate("..")
Pkg.resolve() 
Pkg.instantiate()
using DifferentialEquations,JPEC, Plots, DelimitedFiles
using Base: atan2

  Activating project at `~/Desktop/JPEC`
  No Changes to `~/Desktop/JPEC/Project.toml`
  No Changes to `~/Desktop/JPEC/Manifest.toml`


In [2]:
equil_input = JPEC.Equilibrium.EquilConfig(; control=JPEC.Equilibrium.EquilControl(eq_type="lar", eq_filename="lar.toml"), output=JPEC.Equilibrium.EquilOutput())
lar_input = JPEC.Equilibrium.LargeAspectRatioConfig(; lar_r0=10, lar_a=1, beta0=0.001, q0=1.5, p_pres=2, p_sig=1)

┌ Info: Forcing hamada coordinate jacobian exponents: power_*
└ @ JPEC.Equilibrium /Users/iseonjae/Desktop/JPEC/src/Equilibrium/EquilibriumTypes.jl:48


JPEC.Equilibrium.LargeAspectRatioConfig(10.0, 1.0, 0.001, 1.5, 2.0, 1.0, "default", 128, 128, false)

In [3]:
input= JPEC.Equilibrium.lar_run(equil_input, lar_input)

JPEC.Equilibrium.InverseRunInput(JPEC.Equilibrium.EquilConfig(JPEC.Equilibrium.EquilControl("lar", "lar.toml", "hamada", 0, 0, 0, "ldp", 0.01, 0.994, 128, 256, 0, 1.0e-7, false, false, true), JPEC.Equilibrium.EquilOutput(false, false, false, false, true, false, false, false)), JPEC.SplinesMod.RealSplineType(Ptr{Nothing} @0x0000000123a76e80, [8.682934522190448e-5, 0.0003469834908650275, 0.0007804216571205545, 0.0013869981158246953, 0.00216649771732363, 0.003118641436876229, 0.004243087880758738, 0.005539434077339375, 0.007007216114005258, 0.008645909762575139  …  0.899448974892095, 0.9106051275582959, 0.9217671272924324, 0.9329341766253691, 0.9441054953426861, 0.9552803202460893, 0.9664579049149282, 0.9776375194661567, 0.988818450311505, 1.0000000000000022], [9.999997927148243 0.000499939909263041 1.5000898507093468; 9.999991719551208 0.0004997596587188468 1.5003596562860342; … ; 9.988421733944673 1.1925522940860181e-7 2.9746365776780803; 9.988336160471356 1.7645466826380788e-23 2.99777

In [4]:
config = input.config
rz_in = input.rz_in
sq_in = input.sq_in
ro = input.ro
zo = input.zo
psio = input.psio

grid_type = config.control.grid_type
mpsi = config.control.mpsi
mtheta = config.control.mtheta
psilow = config.control.psilow
psihigh = config.control.psihigh
newq0 = config.control.newq0

me = 3
interp = false
diagnose_rz_in = false
diagnose_rzphi = false


false

In [19]:
local rzphi::Spl.BicubicSplineType
local eqfun::Spl.BicubicSplineType
local rzphi_fs_nodes 

In [20]:
# c-----------------------------------------------------------------------
# c     allocate and define local arrays.
# c-----------------------------------------------------------------------
# sq_in._fs[:, 3] .= sqrt.(sq_in._xs)
rz_in._xs = sq_in._xs
rz_in._ys = collect(0:rz_in.my) ./ rz_in.my

mx = rz_in.mx
my = rz_in.my

x = rz_in.fs[:, :, 1] .- ro
y = rz_in.fs[:, :, 2] .- zo
r2 = x.^2 .+ y.^2

twopi = 2 * pi

deta = zeros(Float64, mx+1, my+1)
for ipsi in 0:mx, itheta in 0:my
    if r2[ipsi+1, itheta+1] == 0.0
        deta[ipsi+1, itheta+1] = 0.0
    else
        deta[ipsi+1, itheta+1] = atan(y[ipsi+1, itheta+1], x[ipsi+1, itheta+1]) / twopi
    end
end

In [25]:
# c-----------------------------------------------------------------------
# c     transform input coordinates from cartesian to polar.
# c-----------------------------------------------------------------------
for ipsi in 0:mx
    for itheta in 1:my
        Δ = deta[ipsi+1, itheta+1] - deta[ipsi+1, itheta]
        if Δ > 0.5
            deta[ipsi+1, itheta+1] -= 1
        elseif Δ < -0.5
            deta[ipsi+1, itheta+1] += 1
        end
    end
    for itheta in 0:my
        if r2[ipsi+1, itheta+1] > 0
            deta[ipsi+1, itheta+1] -= rz_in.ys[itheta+1]
        end
    end
end

deta[1, :] = JPEC.Equilibrium.inverse_extrap(r2[2:me+1, :], deta[2:me+1, :], 0.0)

129-element Vector{Float64}:
  0.0
 -0.023437500000000916
 -0.04687500000000107
 -0.07031250000000182
 -0.09375000000000244
 -0.1171875000000009
 -0.14062499999999942
 -0.16406249999999795
 -0.18750000000000147
 -0.21093750000000305
  ⋮
 -2.8124999999999973
 -2.8359375000000013
 -2.859375
 -2.882812499999999
 -2.9062499999999973
 -2.9296874999999973
 -2.9531249999999973
 -2.9765625
 -3.0000000000000004

In [7]:
# c-----------------------------------------------------------------------
# c     prepare new spline type for surface quantities.
# c-----------------------------------------------------------------------

if grid_type in ["original", "orig"]
    mpsi = size(sq_in._fs, 1) - 1
end

# c-----------------------------------------------------------------------
# c     set up radial grid (only "ldp" implemented)
# c-----------------------------------------------------------------------
if grid_type == "ldp"
    xs = psilow .+ (psihigh - psilow) .* (sin.(range(0.0, 1.0; length=mpsi+1) .* (π/2))).^2
    fs = zeros(Float64, mpsi+1, 4) 
    sq = Spl.spline_setup(xs, fs; bctype="extrap")
else
    error("Only 'ldp' grid_type is implemented for now.")
end

# mtheta가 0이면 rz_in의 my를 사용
if mtheta == 0
    mtheta = rz_in.my
end
rzphi_fs = zeros(Float64, mpsi+1, mtheta+1, 4)
eqfun_fs = zeros(Float64, mpsi+1, mtheta+1, 3)

rzphi_xs = copy(sq.xs)
rzphi_ys = collect(0:mtheta) ./ mtheta
eqfun_xs = copy(sq.xs)
eqfun_ys = collect(0:mtheta) ./ mtheta

rzphi = Spl.bicube_setup(copy(sq.xs), collect(0:mtheta) ./ mtheta, rzphi_fs)
eqfun = Spl.bicube_setup(copy(sq.xs), collect(0:mtheta) ./ mtheta, eqfun_fs)

JPEC.SplinesMod.BicubicSplineType(Ptr{Nothing} @0x000000035580a000, [0.01, 0.010148181201467524, 0.010592635547055184, 0.011333095314084414, 0.012369114477279132, 0.013700068977434681, 0.015325157097327761, 0.017243399944640907, 0.019453642041610618, 0.021954552021043953  …  0.9820454479789561, 0.9845463579583893, 0.986756600055359, 0.9886748429026722, 0.9902999310225652, 0.9916308855227208, 0.9926669046859157, 0.9934073644529449, 0.9938518187985325, 0.994], [0.0, 0.00390625, 0.0078125, 0.01171875, 0.015625, 0.01953125, 0.0234375, 0.02734375, 0.03125, 0.03515625  …  0.96484375, 0.96875, 0.97265625, 0.9765625, 0.98046875, 0.984375, 0.98828125, 0.9921875, 0.99609375, 1.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 128, 256, 3, 4, 4, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.

In [9]:
spl_xs = zeros(Float64,mtheta+1)
spl_fs = zeros(Float64,mtheta+1, 5)
spl = Spl.spline_setup(spl_xs, spl_fs; bctype="extrap")

for ipsi in 0:mpsi
    psifac = rzphi.xs[ipsi+1]
    f_s = Spl.spline_eval(sq, psifac)
    # spl.xs = rzphi.ys
    for itheta in 0:mtheta
        theta = rzphi.ys[itheta+1]
        print(psifac, " ", theta, " ")
        fs_rz_, fsx_rz_, fsy_rz_ = Spl.bicube_eval(rz_, psifac, theta, 1)
        f_sq = Spl.spline_eval(sq_in, psifac, 0)
        if f_rz_[1]<0
            error("Negative radius found at psi=$(psifac), theta=$(theta).")
        end

        rfac = sqrt(f_rz_[1])
        r = ro+rfac*cos(twopi*(theta+fsx_rz_[2]))
        jacfac = fsx_rz_[1] * (1+ fsy_rz_[2]) - fsy_rz_[1] * fsx_rz_[2]
        w11 = (1+ fsy_rz_[2]) * twopi^2 *rfac / jacfac
        w12 = - fsy_rz_[1] * pi/ (rfac * jacfac)
        bp = psio * sqrt(w11^2 + w12^2)/r
        bt = f_sq[1]/r
        b = sqrt(bp^2 + bt^2)

        spl.fs[itheta+1, 1] = f_rz_[1]
        spl.fs[itheta+1, 2] = f_rz_[2]
        spl.fs[itheta+1, 3] = r*jacfac
        spl.fs[itheta+1, 4] = spl.fs[itheta+1, 3]/(r*r)
        spl.fs[itheta+1, 5] = spl.fs[itheta+1, 3]*bp^(config.power_bp)*b^(config.power_b)/r^(config.power_r)
    end

    spl.xs = spl.integral!()
    spl.xs = spl.fsi[:, 5] ./ spl.fs[end, 5]
    spl.fs[:, 2] .= spl.fs[:, 2] .+ rzphi.ys .- spl.xs
    spl.fs[:, 4] .= (spl.fs[:, 3] ./ spl.fsi[end, 3]) ./ (spl.fs[:, 5] ./ spl.fsi[end, 5]) .* spl.fsi[end, 3] * twopi * pi
    spl.fs[:, 3] .= f_sq[1] * pi / psio .* (spl.fsi[:, 4] .- spl.fsi[end, 4] .* spl.xs)
    # spl.xs = Spl.spline_setup(spl.xs, spl.fs; bctype="periodic").integral!()

    for itheta in 0:mtheta
        theta = rzphi.ys[itheta+1]
        f = Spl.spline_eval(spl, theta, 0)
        rzphi.fs[ipsi+1, itheta+1, 1:4] .= f[1:4]
    end

    sq.fs[ipsi+1, 1] = f_sq[ipsi+1, 1] * twopi
    sq.fs[ipsi+1, 2] = f_sq[ipsi+1, 2]
    sq.fs[ipsi+1, 3] = spl.fsi[end, 3] * twopi * pi
    sq.fs[ipsi+1, 4] = spl.fsi[end, 4] * sq.fs[ipsi+1, 1] / (2 * twopi * psio)
end

# sq = Spl.spline_setup(sq.xs, sq.fs; bctype="extrap")
f = Spl.spline_eval(sq, sq.xs[1], 0)
_, f1 = Spl.spline_eval(sq, sq.xs[1], 1)
q0 = f[4] - f1[4] * sq.xs[1]

if newq0 == -1
    newq0 = -q0
end


0.01 0.0 

MethodError: MethodError: no method matching bicube_eval(::Array{Float64, 3}, ::Float64, ::Float64, ::Int64)
The function `bicube_eval` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  bicube_eval(!Matched::JPEC.SplinesMod.BicubicSplineType, ::Any, ::Any, ::Int64)
   @ JPEC ~/Desktop/JPEC/src/Splines/BicubicSpline.jl:219
  bicube_eval(!Matched::JPEC.SplinesMod.BicubicSplineType, ::Any, ::Any)
   @ JPEC ~/Desktop/JPEC/src/Splines/BicubicSpline.jl:219


In [31]:
equilibrium_solver(lar_in)

--- Starting Inverse Equilibrium Processing ---


MethodError: MethodError: no method matching getindex(::JPEC.SplinesMod.BicubicSplineType, ::Symbol)
The function `getindex` exists, but no method is defined for this combination of argument types.

In [ ]:
function save_and_plot_sq_in(sq_in, filename_prefix="sq_in")
    xs = sq_in._xs
    fs = sq_in._fs  # size = (ma+1, 3): [F*Bphi, pressure, q]

    # Save all data to a single txt file
    data = hcat(xs, fs)

    # Plot 1: F*Bphi
    p1 = plot(xs, fs[:,1], label="F*Bphi", xlabel="ψ / ψₒ", ylabel="F*Bphi", lw=2, title="F*Bphi Profile")
    display(p1)

    # Plot 2: Pressure
    p2 = plot(xs, fs[:,2], label="Pressure", xlabel="ψ / ψₒ", ylabel="Pressure", lw=2, title="Pressure Profile")
    display(p2)

    # Plot 3: q
    p3 = plot(xs, fs[:,3], label="q", xlabel="ψ / ψₒ", ylabel="q", lw=2, title="Safety Factor q Profile")
    display(p3)

    println("✅ Displayed plots and saved $filename_prefix.txt")
end

save_and_plot_sq_in(sq_in)